<a href="https://colab.research.google.com/github/azank1/llmshi/blob/main/Copy_of_Lab3_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3:Introduction to RAG

## Task 1: Requirements and Setup

Azan Hyder #365860


- Install the required libraries and ensure GPU access in Google Colab.

In [ ]:
# Download PDF file
import os
import requests

# Get PDF document
pdf_path = "human-nutrition-text.pdf"

# Download PDF if it doesn't already exist
if not os.path.exists(pdf_path):
  print("File doesn't exist, downloading...")

  # The URL of the PDF you want to download
  url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

  # The local filename to save the downloaded file
  filename = pdf_path

  # Send a GET request to the URL
  response = requests.get(url)

  # Check if the request was successful
  if response.status_code == 200:
      # Open a file in binary write mode and save the content to it
      with open(filename, "wb") as file:
          file.write(response.content)
      print(f"The file has been downloaded and saved as {filename}")
  else:
      print(f"Failed to download the file. Status code: {response.status_code}")
else:
  print(f"File {pdf_path} exists.")

File human-nutrition-text.pdf exists.


In [ ]:
# Requires !pip install PyMuPDF, see: https://github.com/pymupdf/pymupdf
import fitz # (pymupdf, found this is better than pypdf for our use case, note: licence is AGPL-3.0, keep that in mind if you want to use any code commercially)
from tqdm.auto import tqdm # for progress bars, requires !pip install tqdm

def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip() # note: this might be different for each doc (best to experiment)

    # Other potential text formatting functions can go here
    return cleaned_text

# Open PDF and get lines/pages
# Note: this only focuses on text, rather than images/figures etc
def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """
    Opens a PDF file, reads its text content page by page, and collects statistics.

    Parameters:
        pdf_path (str): The file path to the PDF document to be opened and read.

    Returns:
        list[dict]: A list of dictionaries, each containing the page number
        (adjusted), character count, word count, sentence count, token count, and the extracted text
        for each page.
    """
    doc = fitz.open(pdf_path)  # open a document
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):  # iterate the document pages
        text = page.get_text() # get the text of the page
        text = text_formatter(text)
        pages_and_texts.append({
            "page_number": page_number - 4, # adjust page number since front matter pages are not always numbered
            "text": text
        })
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': -4, 'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -3, 'text': ''}]

In [ ]:
# Perform Google Colab installs (if running in Google Colab)
import os

if "COLAB_GPU" in os.environ:
    print("[INFO] Running in Google Colab, installing requirements.")
    !pip install -U torch # requires torch 2.1.1+ (for efficient sdpa implementation)
    !pip install PyMuPDF # for reading PDFs with Python
    !pip install tqdm # for progress bars
    !pip install sentence-transformers # for embedding models
    !pip install accelerate # for quantization model loading
    !pip install bitsandbytes # for quantizing models (less storage space)
    # REMOVED flash-attn to prevent build errors

[INFO] Running in Google Colab, installing requirements.


## Task 2: Import PDF Document

- Import the PDF file and extract its pages using PyMuPDF.

In [ ]:
# Download PDF file
import os
import requests

# Get PDF document
pdf_path = "human-nutrition-text.pdf"

# Download PDF if it doesn't already exist
if not os.path.exists(pdf_path):
  print("File doesn't exist, downloading...")

  # The URL of the PDF you want to download
  url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

  # The local filename to save the downloaded file
  filename = pdf_path

  # Send a GET request to the URL
  response = requests.get(url)

  # Check if the request was successful
  if response.status_code == 200:
      # Open a file in binary write mode and save the content to it
      with open(filename, "wb") as file:
          file.write(response.content)
      print(f"The file has been downloaded and saved as {filename}")
  else:
      print(f"Failed to download the file. Status code: {response.status_code}")
else:
  print(f"File {pdf_path} exists.")

File human-nutrition-text.pdf exists.


In [ ]:
# Requires !pip install PyMuPDF, see: https://github.com/pymupdf/pymupdf
import fitz # (pymupdf, found this is better than pypdf for our use case, note: licence is AGPL-3.0, keep that in mind if you want to use any code commercially)
from tqdm.auto import tqdm # for progress bars, requires !pip install tqdm

def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip() # note: this might be different for each doc (best to experiment)

    # Other potential text formatting functions can go here
    return cleaned_text

# Open PDF and get lines/pages
# Note: this only focuses on text, rather than images/figures etc
def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """
    Opens a PDF file, reads its text content page by page, and collects statistics.

    Parameters:
        pdf_path (str): The file path to the PDF document to be opened and read.

    Returns:
        list[dict]: A list of dictionaries, each containing the page number
        (adjusted), character count, word count, sentence count, token count, and the extracted text
        for each page.
    """
    doc = fitz.open(pdf_path)  # open a document
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):  # iterate the document pages
        text = page.get_text() # get the text of the page
        text = text_formatter(text)
        pages_and_texts.append({
            "page_number": page_number - 4, # adjust page number since front matter pages are not always numbered
            "text": text
        })
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': -4, 'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -3, 'text': ''}]

## Task 3: Sample Pages

- Select and display a random sample of pages from the document.

In [ ]:
import random

random.sample(pages_and_texts, k=3)

[{'page_number': 918,
  'text': 'example. Children who suffer from this condition experience an  adverse reaction to the lactose in milk products. It is a result of  the small intestine’s inability to produce enough of the enzyme  lactase, which is produced by the small intestine. Symptoms of  lactose intolerance usually affect the GI tract and can include  bloating, abdominal pain, gas, nausea, and diarrhea. An intolerance  is best managed by making dietary changes and avoiding any foods  that trigger the reaction.8  The Threat of Lead Toxicity  There is a danger of lead toxicity, or lead poisoning, among school- aged children. Lead is found in plumbing in old homes, in lead-based  paint, and occasionally in the soil. Contaminated food and water can  increase exposure and result in hazardous lead levels in the blood.  Children under age six are especially vulnerable. They may consume  items tainted with lead, such as chipped, lead-based paint. Another  common exposure is lead dust in 

## Task 4: Text Statistics

- Compute and display descriptive statistics (e.g., character counts, word counts) of the text pages.

In [ ]:
import pandas as pd

# Create DataFrame from pages_and_texts
df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,text
0,-4,Human Nutrition: 2020 Edition
1,-3,
2,-2,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-1,Human Nutrition: 2020 Edition by University of...
4,0,Contents Preface University of Hawai‘i at Mā...


In [ ]:
# Get stats
df.describe().round(2)

,page_number
count,1208.00
mean,599.50
std,348.86
min,-4.00
25%,297.75
50%,599.50
75%,901.25
max,1203.00


## Task 5: Sentence Tokenization

- Split the text pages into sentences using spaCy or an equivalent library.

In [ ]:
from spacy.lang.en import English # see https://spacy.io/usage for install instructions

nlp = English()

# Add a sentencizer pipeline, see https://spacy.io/api/sentencizer/
nlp.add_pipe("sentencizer")

# Create a document instance as an example
doc = nlp("This is a sentence. This is another sentence.")

# Access the sentences of the document
list(doc.sents)

[This is a sentence., This is another sentence.]

In [ ]:
from tqdm.auto import tqdm # Import tqdm
for item in tqdm(pages_and_texts):
    item["sentences"] = [str(sent) for sent in list(nlp(item["text"]).sents)]

    # Count the sentences
    item["num_sentences"] = len(item["sentences"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [ ]:
import random # Import random
# Inspect an example
random.sample(pages_and_texts, k=1)

[{'page_number': 789,
  'text': 'given based on the amount consumed. For example, a score of  ten is given if a 2,000-kilocalorie diet includes greater than 2.6  cups of milk per day. If less than 10 percent of total calories in  a diet are from saturated fat, a score of eight is given. All of the  scores are added up from the different food categories and the  diets are given a HEI score. Using this standardized diet-assessment  tool at different times, every ten years for instance, the CNPP can  determine if the eating habits of certain groups of the American  population are getting better or worse. The HEI tool provides the  federal government with information to make policy changes to  better the diets of American people. For more information on the  HEI,  visit  this  website:  http://www.cnpp.usda.gov/ healthyeatingindex.htm.  The Whole Nutrient Package versus Disease  A healthy diet incorporating seven or more servings of fruits and  vegetables has been shown in many scientific 

In [ ]:
import pandas as pd
df = pd.DataFrame(pages_and_texts)
display(df.describe().round(2))

,page_number,num_sentences
count,1208.00,1208.00
mean,599.50,10.32
std,348.86,6.30
min,-4.00,0.00
25%,297.75,5.00
50%,599.50,10.00
75%,901.25,15.00
max,1203.00,28.00


## Task 6: Chunk Sentences

- Group sentences into chunks of a specified size.

In [ ]:
# Define split size to turn groups of sentences into chunks
num_sentence_chunk_size = 10

# Create a function that recursively splits a list into desired sizes
def split_list(input_list: list,
               slice_size: int) -> list[list[str]]:
    """
    Splits the input_list into sublists of size slice_size (or as close as possible).

    For example, a list of 17 sentences would be split into two lists of [[10], [7]]
    """
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [ ]:
import random
# Sample an example from the group (note: many samples have only 1 chunk as they have <=10 sentences total)
random.sample(pages_and_texts, k=1)

[{'page_number': 621,
  'text': 'Image by  Allison  Calabrese /  CC BY 4.0  Vitamin B12 and folate play key roles in converting homocysteine  to amino acid methionine. \xa0As mentioned in Figure 9.19 “ Vitamin  B6 Functional Coenzyme Role”, high levels of homocysteine in the  blood increases the risk for heart disease. Low levels of vitamin  B12, folate or vitamin B6 will increase homocysteine levels therefore  increasing the risk of heart disease.  Figure 9.20 The Relationship Between Folate and Vitamin B12  When there is a deficiency in vitamin B12 , inactive folate (from  food) is unable to be converted to active folate and used in the  body for the synthesis of DNA. Folic Acid however (that comes from  supplements or fortified foods) is available to be used as active  folate in the body without vitamin B12 .Therefore, if there is a  deficiency in vitamin B12 macrocytic anemia may occur. With the  fortification of foods incorporated into people’s diets, the risk of an  individual de

In [ ]:
import pandas as pd
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,num_sentences,num_chunks
count,1208.00,1208.00,1208.00
mean,599.50,10.32,1.53
std,348.86,6.30,0.64
min,-4.00,0.00,0.00
25%,297.75,5.00,1.00
50%,599.50,10.00,1.00
75%,901.25,15.00,2.00
max,1203.00,28.00,3.00


## Task 7: Split Chunks

- Split each sentence chunk into individual records for embedding preparation.

In [ ]:
import re

# Split each chunk into its own item
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
        joined_sentence_chunk = " ".join(sentence_chunk)
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get stats about the chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(nlp(joined_sentence_chunk)) # Using spaCy's tokenizer for token count

        pages_and_chunks.append(chunk_dict)

# How many chunks do we have?
len(pages_and_chunks)

  0%|          | 0/1208 [00:00<?, ?it/s]

1843

In [ ]:
import random # Import random, if not already imported
# View a random sample
random.sample(pages_and_chunks, k=1)

[{'page_number': 860,
  'sentence_chunk': 'inhibits milk secretion. Shortly after birth, the expulsion of the  placenta triggers progesterone levels to fall, which activates  lactation.15 When the infant suckles at the breast, levels of the  hormone oxytocin rise to promote the release of breast milk from  the breast when the infant suckles, which is known as the milk- ejection reflex.  New mothers usually find that their appetite and thirst is greater  than before pregnancy; it is recommended that they still focus on  nutrient-dense foods to nourish their body and replace their body’s  nutrient stores. A conservative rate of weight loss (1-2 pounds per  week) during lactation does not usually impact the quantity or  quality of breast milk, but maternal deficiencies in some nutrients  have been described during lactation. The nutrient content of  breastmilk does not change much based upon maternal diet for  most nutrients. The RDA for energy is 330 additional Calories during  the first

In [ ]:
import pandas as pd # Import pandas, if not already imported
# Get stats about our chunks
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1843.00,1843.00,1843.00,1843.00
mean,620.38,752.60,130.45,151.01
std,347.79,456.15,80.44,91.86
min,-4.00,15.00,4.00,5.00
25%,317.50,323.00,54.00,66.00
50%,623.00,766.00,134.00,155.00
75%,927.00,1140.00,197.00,227.00
max,1203.00,1871.00,413.00,455.00


## Task 8: Filter Small Chunks

- Filter out text chunks below a minimum token length threshold.

In [ ]:
# Show random chunks with under 30 tokens in length
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 26 | Text:  Health- calc.com  offers  an  interactive  TEE  calculator.  http://www.health-calc.com/diet/energy- expenditure-advanced  482  |  Weight Management
Chunk token count: 29 | Text: 82. http://www.cdc.gov/ nchs/data/databriefs/db82.pdf. Published January 2012.  Accessed December 8, 2017.  Young Adulthood  |  911
Chunk token count: 12 | Text: US Department of Agriculture,  1136  |  Food Insecurity
Chunk token count: 25 | Text: http://www.heart.org/HEARTORG/ HealthyLiving/PhysicalActivity/FitnessBasics/ American-Heart-Association-Recommendations-for- 936  |  The Essential Elements of Physical Fitness
Chunk token count: 14 | Text:  Health Consequences and Benefits of High-Carbohydrate Diets  |  267


In [ ]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': -2,
  'sentence_chunk': 'Human Nutrition: 2020  Edition  UNIVERSITY OF HAWAI‘I AT MĀNOA  FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM  ALAN TITCHENAL, SKYLAR HARA,  NOEMI ARCEO CAACBAY, WILLIAM  MEINKE-LAU, YA-YUN YANG, MARIE  KAINOA FIALKOWSKI REVILLA,  JENNIFER DRAPER, GEMADY  LANGFELDER, CHERYL GIBBY, CHYNA  NICOLE CHUN, AND ALLISON  CALABRESE',
  'chunk_char_count': 320,
  'chunk_word_count': 54,
  'chunk_token_count': 69},
 {'page_number': -1,
  'sentence_chunk': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and  Human Nutrition Program is licensed under a Creative Commons Attribution 4.0  International License, except where otherwise noted.',
  'chunk_char_count': 212,
  'chunk_word_count': 32,
  'chunk_token_count': 35}]

## Task 9: Embed Text Chunks

- Embed each text chunk into numeric vectors using a sentence-transformer model.

In [ ]:
# Install necessary library if not already present
!pip install sentence-transformers -q

from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device="cuda") # choose the device to load the model to (note: GPU will often be *much* faster than CPU)

# Create a list of sentences to turn into numbers
sentences = [
    "The Sentences Transformers library provides an easy and open-source way to create embeddings.",
    "Sentences can be embedded one by one or as a list of strings.",
    "Embeddings are one of the most powerful concepts in machine learning!",
    "Learn to use embeddings well and you'll be well on your way to being an AI engineer."
]

# Sentences are encoded/embedded by calling model.encode()
embeddings = embedding_model.encode(sentences)
embeddings_dict = dict(zip(sentences, embeddings))

# See the embeddings
for sentence, embedding in embeddings_dict.items():
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

ModuleNotFoundError: Could not import module 'PreTrainedModel'. Are this object's requirements defined correctly?

In [ ]:
single_sentence = "Yo! How cool are embeddings?"
single_embedding = embedding_model.encode(single_sentence)
print(f"Sentence: {single_sentence}")
print(f"Embedding:\n{single_embedding}")
print(f"Embedding size: {single_embedding.shape}")

## Task 10: Batch Embedding on GPU

- Execute batched embeddings on GPU to improve performance and measure speed.

In [ ]:
# Turn text chunks into a single list
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

In [ ]:
%%time

# Embed all texts in batches
text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=32,
                                               convert_to_tensor=False)

print(f"Shape: {text_chunk_embeddings.shape}")

## Task 11: Save and Load Embeddings

- Save the embeddings to a CSV file and reload it to verify integrity.

In [ ]:
# Save embeddings to file
embeddings_df_save_path = "text_chunks_and_embeddings.csv"

# Create a DataFrame to save
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
text_chunks_and_embeddings_df["embedding"] = list(text_chunk_embeddings)

# Save to CSV
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [ ]:
# Import saved file and view
import pandas as pd
import numpy as np

text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)

# NOTE: When loading from CSV, embeddings are strings.
# For this session, we will re-assign the list of arrays from memory to ensuring they work with FAISS.
df = text_chunks_and_embedding_df_load
df["embedding"] = list(text_chunk_embeddings)

df.head()

## Task 12: Build Retrieval System

- **Load embeddings**  
  Read `text_chunks_and_embeddings_df.csv` into a pandas DataFrame.

- **Create a vector index**  
  Initialize a FAISS (or similar) index with the chunk embeddings.

- **Index all chunks**  
  Add every embedding vector to your index to prepare for similarity search.

- **Implement a retrieval function** that:
  1. Takes a user query (string) as input.  
  2. Embeds it using the same sentence‑transformer model.  
  3. Runs a top‑k similarity search (e.g. k = 5) against your FAISS index.  
  4. Returns the corresponding text chunks.

- **Test your pipeline**  
  Try queries like “What is the recommended daily protein intake?” and inspect the retrieved passages.



In [ ]:
#Load the CSV of chunks + embeddings


# 1. Load the same model you used for embedding


# 2. Embed all sentence chunks in batches


# 3. Attach embeddings back to DataFrame


In [ ]:
!pip install faiss-cpu -q
import faiss
import numpy as np

# 1. Get embedding dimension
dim = df["embedding"].iloc[0].shape[0]

# 2. Create a flat L2 index
index = faiss.IndexFlatL2(dim)

# 3. Add all embeddings to the index
# Stack embeddings into a matrix for FAISS
embeddings_matrix = np.vstack(df["embedding"].to_list())
index.add(embeddings_matrix)
print("FAISS index: n_vectors =", index.ntotal)

In [ ]:
def retrieve(query: str, k: int = 5):
    # 1. Embed the query
    # Note: ensure we use the same 'embedding_model' from earlier
    q_emb = embedding_model.encode(query).astype("float32").reshape(1, -1)

    # 2. Search for top-k
    dists, idxs = index.search(q_emb, k)

    # 3. Collect results
    return [
        {
            "page": int(df.iloc[i]["page_number"]),
            "chunk": df.iloc[i]["sentence_chunk"],
            "distance": float(d)
        }
        for d, i in zip(dists[0], idxs[0])
    ]

# Example usage
# Query for 'protein' to verify Task 12 checklist
results = retrieve("What is the recommended daily intake of protein?")
for hit in results:
    print(f"[Pg {hit['page']}] (dist={hit['distance']:.3f}) {hit['chunk']}")

if results:
    print("\n[SUCCESS] Retrieval system is functioning correctly.")
else:
    print("\n[WARNING] No results found.")

## Task 13: Questions

- Answer the following questions regarding embedding model selection and chunking:

1. Which embedding model should you use and why?

2. How might chunk size affect embedding performance?

3. Why is chunking necessary in a RAG pipeline, and what would happen if we embedded entire documents instead?

4. Why must the same embedding model be used for both document chunks and user queries?

5. Why does this lab use vector similarity search (FAISS) instead of keyword matching or rule-based search?

6. Can a RAG system still return incorrect or misleading answers even if retrieval works correctly? Explain why.

# Task
Install the required libraries (`torch`, `PyMuPDF`, `tqdm`, `sentence-transformers`, `accelerate`, `bitsandbytes`, and `flash-attn`) and verify that all installations are successful and the environment is ready for further tasks, including confirming GPU access.

## Install Requirements

### Subtask:
Execute the code cell to install the required libraries like `torch`, `PyMuPDF`, `tqdm`, `sentence-transformers`, `accelerate`, `bitsandbytes`, and `flash-attn`. This step will also check for GPU access.


**Reasoning**:
The subtask requires executing the provided code cell to install necessary libraries and check for GPU access. The code cell `277c519c` performs exactly this action.



In [ ]:
# Perform G
import os

if "COLAB_GPU" in os.environ:
    print("[INFO] Running in Google Colab, installing requirements.")
    !pip install -U torch # requires torch 2.1.1+ (for efficient sdpa implementation)
    !pip install PyMuPDF # for reading PDFs with Python
    !pip install tqdm # for progress bars
    !pip install sentence-transformers # for embedding models
    !pip install accelerate # for quantization model loading
    !pip install bitsandbytes # for quantizing models (less storage space)
    !pip install flash-attn --no-build-isolation # for faster attention mechanism = faster LLM inference

**Reasoning**:
The subtask explicitly states to execute the provided code cell to install necessary libraries and check for GPU access. The code cell `277c519c` is designed for this purpose.



In [ ]:
import os

if "COLAB_GPU" in os.environ:
    print("[INFO] Running in Google Colab, installing requirements.")
    !pip install -U torch # requires torch 2.1.1+ (for efficient sdpa implementation)
    !pip install PyMuPDF # for reading PDFs with Python
    !pip install tqdm # for progress bars
    !pip install sentence-transformers # for embedding models
    !pip install accelerate # for quantization model loading
    !pip install bitsandbytes # for quantizing models (less storage space)
    !pip install flash-attn --no-build-isolation # for faster attention mechanism = faster LLM inference